In [1]:
import os
import shutil

# = = = = = = = = = = = = = = =

if os.path.exists('./file/4.collate'):
    shutil.rmtree('./file/4.collate')

os.makedirs('./file/4.collate')

# 1. Json

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.oem'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/3.oem/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Rating Count'] = input_['Rating Count'].astype(int)
    
    # = = = = = = = = = = = = = = =

    list_column, list_column_description = input_.columns.tolist(), []

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
        df_temp, dict_dict = pd.DataFrame([input_.iloc[i]]).reset_index(drop=True), json.loads(input_.loc[i, 'Json_Description'])

        for dict_ in dict_dict.values():
            for key, value in dict_.items():
                if key in list_column:
                    key = f'''{key} (Description)'''
                
                if key not in list_column_description:
                    list_column_description.append(key)

                if type(value) == dict:
                    list_value = []
                    for key_2, value_2 in value.items():
                        list_value.append(f'''{int(key_2) + 1}. {value_2}''')
                    df_temp.loc[0, key] = '\n'.join(list_value)
                else:
                    df_temp.loc[0, key] = value
    
        list_df.append(df_temp)
    
    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output[list_column[:10] + list_column[13:] + sorted(list_column_description) + list_column[10:13]].to_excel(f'./file/4.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 297.96it/s]


crawler_2.xlsx



Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 867.00it/s]


Done ~


# 2. Compatible

In [3]:
import pandas as pd

from tqdm import tqdm

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/3.vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Vehicle No'] = input_['Vehicle No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)

    # = = = = = = = = = = = = = = =

    input_2 = input_.drop_duplicates(['OEM Url'],
                                     keep='first',
                                     ignore_index=True)
    input_2 = input_2.sort_values(by=['No'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp = input_[input_['OEM Url'] == input_2.loc[i, 'OEM Url']].reset_index(drop=True)

        dict_vehicle = {}
        for j in range(len(df_temp)):
            make, model, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Year']
    
            if make not in dict_vehicle:
                dict_vehicle[make] = {model: [year]}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            elif year not in dict_vehicle[make][model]:
                dict_vehicle[make][model].append(year)
    
        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make in dict_vehicle:
            for model in dict_vehicle[make]:
                year_begin = min(dict_vehicle[make][model])
                year_end = max(dict_vehicle[make][model])
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        df_temp = pd.DataFrame([{'No': input_2.loc[i, 'No'],
                                 'SKU': input_2.loc[i, 'SKU'],
                                 'OEM Url': input_2.loc[i, 'OEM Url'],
                                 'Vehicle': '\n'.join(sorted(list_vehicle))}])
        list_df.append(df_temp)

    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output.to_excel(f'./file/4.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 807.82it/s]



crawler_2-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 812.61it/s]


Done ~
